<a href="https://colab.research.google.com/github/bozorgpanah/The-Explainable-Machine-Learning-Model-withPrivacy/blob/main/Paper1/MDAV%2BSHAP%2B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Split matrices into random train and test subsets_Devided data set in two parts include train and test parts 70%-30% respectively
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_classification
import scipy
%matplotlib inline
from google.colab import files
import seaborn as sn

X, y = make_classification(n_samples=100, n_features=10, n_redundant=3, n_repeated=2, 
                           n_informative=5, n_clusters_per_class=4, 
                           random_state=42) #for reproducibility 

plt.figure(figsize=(50,50))
plt.subplots_adjust(bottom=.05, top=.9, left=.05, right=.95)

plt.subplot(321)
plt.title("Five informative feature, Four clusters per class", fontsize='small')

#print (X ,y)
plt.scatter(X[:, 0], X[:, 1], marker='s', c=y, s=25, edgecolor='c')
plt.show()


#Making file to save data
fX = open("Dataset_X.txt","wt")
fX.write("\n"+str(X)+"\n")
fy = open("Dataset_y.txt","wt")
fy.write("\n"+str(y)+"\n")
fX.close()
fy.close()
files.download("Dataset_X.txt")
files.download("Dataset_y.txt")

z = X.mean()
print (z)
covMatrix = np.cov(X)
'''
sn.heatmap(covMatrix, annot=True, fmt='g')
plt.show()
'''
print(covMatrix)
#Making file to save data
CovRealData = open("covMatrix_RealData.txt","wt")
CovRealData.write("\n"+str(covMatrix)+"\n")
CovRealData.close()
files.download("covMatrix_RealData.txt")


In [ ]:
#Split matrices into random train and test subsets_Devided data set in two parts include train and test parts 70%-30% respectively
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#training data
print (X_train ,y_train)
plt.scatter(X_train[:, 0], X_train[:, 1], marker='s', c=y_train, s=25, edgecolor='c')
plt.show()

#test data
print (X_test ,y_test)
plt.scatter(X_test[:, 0], X_test[:, 1], marker='s', c=y_test, s=25, edgecolor='c')
plt.show()

In [ ]:
#MDAV a microagregation method
def dist(x,y):
    return np.linalg.norm(x-y)
    
#Stack arrays in sequence vertically (row wise)
def poprow(arr,i):
    pop = arr[i]
    new_array = np.vstack((arr[:i],arr[i+1:]))
    return new_array,pop

def cluster(X, p, k, dist_to_xr):
    c = [p]
    
    if dist_to_xr == None:
        distances = [dist(v[:-1],p[:-1]) for v in X]
    else:
        distances = dist_to_xr
        
    X = X[np.argpartition(distances, k-1)]
    c.extend(X[:k-1])
    X = X[k-1:]
    
    xc = np.array([p[:-1] for p in c], copy=False, ndmin=2)
    yc = np.array([p[-1] for p in c], copy=False)
    cl = (xc, yc)
    
    return X, cl

def mdav(X, y, k):
    D = np.column_stack((X,y)) #D is a matrix of variables
    clusters = []
    centroids = []
    while len(D) >= 3*k:
        # Centroid
        xm = np.mean(D, axis=0) #xm is an array includes mean in each variables (each columns)
        # Furthest from centroid
        xri = np.argmax([dist(v[:-1],xm[:-1]) for v in D])#Find furthest from the centroid in each vector and it's called xri
        D, xr = poprow(D, xri)
        # Furthest from furthest from centroid
        dist_to_xr = [dist(v[:-1],xr[:-1]) for v in D]
        xsi = np.argmax(dist_to_xr)
        dist_to_xr = dist_to_xr[:xsi]+dist_to_xr[xsi+1:]
        D, xs = poprow(D, xsi) 

        #cluster of xr
        D, c = cluster(D, xr, k, dist_to_xr)
        clusters.append(c)
        #cluster of xs
        D, c = cluster(D, xs, k, None)
        clusters.append(c)
        
    if len(D) >= 2*k and len(D) < 3*k:
        # Centroid
        xm = np.mean(D, axis=0)
        # Furthest from centroid
        xri = np.argmax([dist(v[:-1],xm[:-1]) for v in D])
        D, xr = poprow(D, xri)
        #cluster of xr
        D, c = cluster(D, xr, k, None)
        clusters.append(c)
        
        # rest of points
        xc = np.array([p[:-1] for p in D[:]], copy=False, ndmin=2)
        yc = np.array([p[-1] for p in D[:]], copy=False)
        cl = (xc, yc)
        clusters.append(cl)     
    else:
        # rest of points
        xc = np.array([p[:-1] for p in D[:]], copy=False, ndmin=2)
        yc = np.array([p[-1] for p in D[:]], copy=False)
        cl = (xc, yc)
        clusters.append(cl)
    
    centroids = np.array([np.mean(c[0],axis=0) for c in clusters], copy=False)
    
    return clusters, centroids

In [ ]:
#11022020 We use smaller K amount
import time
import statistics 
exec_times = []

# Generate clusters for different representativities
#representativity = [0.001,0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 
#                    0.009, 0.01, 0.02, 0.03]
representativity = [0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09,0.1]
K = [int(len(X)*r) for r in representativity]#K = [int(len(X_train)*r) for r in representativity]
clusterings = []
centroids = []
maskedData = []
print(f'k = {K}')

for k in K:
    start = time.time()
    #print(k, K)
    clustering, centroid = mdav(X, y, k)#clustering, centroid = mdav(X_train, y_train, k)
    clusterings.append(clustering)
    centroids.append(centroid) 
    i = 0
    while i<k:
     maskedData.append(centroid) 
     i += 1  

    end = time.time()
    exec_times.append(end-start)
    #arr_ctr_num.append(len(centroids))
    
#print(f'Time for calculating = {exec_times}')

#Saveing centroids in a file
f_centroids = open("Centroids.txt","wt")
f_centroids.write("\n"+str(centroids)+"\n")
f_clusters = open("Clusers.txt","wt")
f_clusters.write("\n"+str(clusterings)+"\n")
f_maskedData = open("MaskedData.txt","wt")
f_maskedData.write("\n"+str(maskedData)+"\n")
f_maskedData.close()
f_centroids.close()
f_clusters.close()
files.download("MaskedData.txt")
files.download("Centroids.txt")
files.download("Clusers.txt")
#dr = np.array([4,2,1,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0,0])
#k = np.array([8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48 , 52])